# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'timestamp': '2018-07-01T00:00:09Z', 'sentiment': 2, 'message_body': '$FITB great buy at 26.00...ill wait'}, {'timestamp': '2018-07-01T00:00:42Z', 'sentiment': 1, 'message_body': '@StockTwits $MSFT'}, {'timestamp': '2018-07-01T00:01:24Z', 'sentiment': 2, 'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating'}, {'timestamp': '2018-07-01T00:01:47Z', 'sentiment': 1, 'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.'}, {'timestamp': '2018-07-01T00:02:13Z', 'sentiment': 0, 'message_body': '$AMD reveal yourself!'}, {'timestamp': '2018-07-01T00:03:10Z', 'sentiment': 1, 'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?'}, {'timestamp': '2018-07-01T00:04:09Z', 'sentiment': -2, 'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
print(messages[:5])
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]
print(sentiments[:5])

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', '$AMD reveal yourself!']
[4, 3, 4, 3, 2]


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')
import re

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http[s]*://[\S]*', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$[A-Za-z]+', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@[\S]+', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^A-Za-z]+', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    text = text.split('\n') # In case there is new line in the text
    text = ' '.join(text)
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.

    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(word, pos="v") for word in tokens if len(word)>1 ] 
    
    return tokens


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leehongkai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement

# define traverse to unlist the lists in a list
def traverse(o, tree_types=(list, tuple)):
    if isinstance(o, tree_types):
        for value in o:
            for subvalue in traverse(value, tree_types):
                yield subvalue
    else:
        yield o

tokenized = [preprocess(message) for message in messages]
tokenized = list(traverse(tokenized))
tokenized[:10]

['great',
 'buy',
 'at',
 'ill',
 'wait',
 'staanalystalert',
 'for',
 'jefferies',
 'maintain',
 'with']

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = Counter(tokenized)
sorted_vocab_raw = sorted(bow, key=bow.get, reverse=True)
bow = {word: bow[word] for word in sorted_vocab_raw}

# bow = Counter()
# for t in tokenized:
#     bow.update(t)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionary that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_count = len(tokenized)
freqs = {word: count/total_count for word, count in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 1e-5

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 12

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = sorted_vocab_raw[:12]

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 


['be', 'the', 'to', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will']


4086

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [40]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: ii for ii, word in enumerate(filtered_words, 1)}
print(vocab)
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii: word for ii, word in vocab.items()}

# tokenized with the words not in `filtered_words` removed.
tokenized = [preprocess(message) for message in messages]
filtered = [[val for val in ll if val in vocab] for ll in tokenized]
len(vocab)

{'eps': 3849, 'gamers': 3833, 'monopoly': 2556, 'dollar': 3569, 'everyone': 2432, 'afford': 1671, 'green': 3718, 'long': 741, 'puke': 2433, 'patient': 997, 'wks': 1545, 'criminal': 3850, 'oracle': 140, 'add': 743, 'rofl': 3749, 'offer': 998, 'dust': 3296, 'dime': 2327, 'typical': 1136, 'milk': 1411, 'changer': 2434, 'pmest': 817, 'turn': 744, 'lrcx': 2786, 'diff': 1547, 'cliff': 3025, 'curve': 1960, 'struggle': 2557, 'hynix': 261, 'ir': 641, 'subscribers': 3458, 'expense': 3595, 'dead': 2902, 'rekt': 2086, 'close': 753, 'elsewhere': 1588, 'onto': 3937, 'bipolar': 1318, 'literally': 3162, 'la': 262, 'apparel': 2565, 'folks': 745, 'lose': 1412, 'corrupt': 3724, 'speech': 3638, 'bottom': 1555, 'rare': 1548, 'recent': 141, 'dirty': 3633, 'anymore': 3820, 'putin': 2957, 'stabilize': 2787, 'pick': 726, 'emotions': 1276, 'ten': 142, 'erase': 3298, 'arent': 1557, 'survey': 1000, 'hot': 263, 'move': 1549, 'software': 2867, 'breathe': 676, 'yday': 3353, 'opinion': 856, 'operations': 1, 'care': 4

4086

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [ ]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [ ]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

Finally let's convert our tokens into integer ids which we can pass to the network.

In [12]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [10]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)
        
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # linear and log softmax layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        
        

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())

        return hidden_state


    
    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        # TODO Implement 
        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        lstm_out = lstm_out[-1,:,:]
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # log softmax function
        logps = self.log_softmax(out)
        
        # return last log softmax output and hidden state
        return logps, hidden_state

No GPU available, training on CPU.


### View Model

In [11]:
# set the "set_default_tensor_type" to "torch.cuda.FloatTensor"
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.3, lstm_layers=2)
model.to(device)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)
print(model)
logps, _ = model.forward(input, hidden)
print(logps)


TextClassifier(
  (embedding): Embedding(4086, 10)
  (lstm): LSTM(10, 6, num_layers=2, dropout=0.3)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)
tensor([[-1.2510, -2.0232, -2.0472, -1.5214, -1.4523],
        [-1.3035, -1.9820, -2.0167, -1.4380, -1.5135],
        [-1.2960, -1.9648, -2.0265, -1.4662, -1.4978],
        [-1.2661, -1.9721, -2.0007, -1.4735, -1.5393]],
       grad_fn=<LogSoftmaxBackward>)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [12]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = batch_size*int(len(messages)/batch_size)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [45]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.8

## split data into training and validation data (features and labels)

split_idx = int(len(token_ids)*split_frac)

train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

## print out the shapes of your resultant feature data
print("Train set: \t\t{}".format(len(train_features)), 
      "\nValidation set: \t{}".format(len(valid_features)))

train_features[:5]
train_labels[:5]

Train set: 		821298 
Validation set: 	205325


[4, 4, 3, 3, 0]

In [46]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.3)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

/Users/leehongkai/anaconda/envs/trading/lib/python3.5/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


### Training
It's time to train the neural network!

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(4087, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [19]:
# """
# Train your model with dropout. Make sure to clip your gradients.
# Print the training loss, validation loss, and validation accuracy for every 100 steps.
# """

# epochs = 2
# batch_size = 512
# learning_rate = 0.05
# clip = 5

# print_every = 100
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# model.train()

# for epoch in range(epochs):
#     print('Starting epoch {}'.format(epoch + 1))
    
#     steps = 0
#     hidden = model.init_hidden(batch_size=batch_size)

#     for text_batch, labels in dataloader(
#             train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
#         steps += 1
        
#         # Set Device
#         text_batch, labels = text_batch.to(device), labels.to(device)
#         for each in hidden:
#             each.to(device)
     
#         # TODO Implement: Train Model
#         # Creating new variables for the hidden state, otherwise
#         # we'd backprop through the entire training history
#         hidden = tuple([each.data for each in hidden])
        
#         # zero accumulated gradients
#         model.zero_grad()

#         # get the output from the model
#         output, hidden = model(text_batch, hidden)
        
#         # calculate the loss and perform backprop
#         loss = criterion(output.squeeze(), labels.long())
#         loss.backward()
        
#         # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
#         nn.utils.clip_grad_norm_(model.parameters(), clip)
#         optimizer.step()
        
#         if steps % print_every == 0:
#             model.eval()
            
#             # TODO Implement: Print metrics
#             print("Epoch: {}/{}...".format(epoch+1, epochs),
#                   "Step: {}...".format(steps),
#                   "Loss: {:.6f}...".format(loss.item()))
            
#             model.train()


Starting epoch 1
Epoch: 1/2... Step: 100... Loss: 1.371718...
Epoch: 1/2... Step: 200... Loss: 1.276035...
Epoch: 1/2... Step: 300... Loss: 1.280844...
Epoch: 1/2... Step: 400... Loss: 1.237524...
Epoch: 1/2... Step: 500... Loss: 1.236321...
Epoch: 1/2... Step: 600... Loss: 1.207283...
Epoch: 1/2... Step: 700... Loss: 1.283078...
Epoch: 1/2... Step: 800... Loss: 1.287650...
Epoch: 1/2... Step: 900... Loss: 1.266801...
Epoch: 1/2... Step: 1000... Loss: 1.298173...
Epoch: 1/2... Step: 1100... Loss: 1.182030...
Epoch: 1/2... Step: 1200... Loss: 1.240853...
Epoch: 1/2... Step: 1300... Loss: 1.248655...
Epoch: 1/2... Step: 1400... Loss: 1.261769...
Epoch: 1/2... Step: 1500... Loss: 1.219162...
Epoch: 1/2... Step: 1600... Loss: 1.352841...
Starting epoch 2
Epoch: 2/2... Step: 100... Loss: 1.225981...
Epoch: 2/2... Step: 200... Loss: 1.234021...
Epoch: 2/2... Step: 300... Loss: 1.267791...
Epoch: 2/2... Step: 400... Loss: 1.150922...
Epoch: 2/2... Step: 500... Loss: 1.256547...
Epoch: 2/2... 

In [48]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
embedding.weight 	 torch.Size([4087, 1024])
lstm.weight_ih_l0 	 torch.Size([2048, 1024])
lstm.weight_hh_l0 	 torch.Size([2048, 512])
lstm.bias_ih_l0 	 torch.Size([2048])
lstm.bias_hh_l0 	 torch.Size([2048])
lstm.weight_ih_l1 	 torch.Size([2048, 512])
lstm.weight_hh_l1 	 torch.Size([2048, 512])
lstm.bias_ih_l1 	 torch.Size([2048])
lstm.bias_hh_l1 	 torch.Size([2048])
fc.weight 	 torch.Size([5, 512])
fc.bias 	 torch.Size([5])


In [13]:
# Saving the model for later prediction use without retraining the model
SAVE_MODEL_PATH = os.path.join('data', 'project_6_stocktwits', 'stocktwits_udacity_model_state_dict')

## It is necessary to switch the model to cpu mode before saving if the model 
## will be loaded for cpu use later
# model.eval()
# model.to('cpu')

## Saving the model state dict
# torch.save(model.state_dict(), SAVE_MODEL_PATH)

# Setting the model parameters according to trained model
model = TextClassifier(4086+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to('cpu')

# Relaoding the model state dict
model.load_state_dict(torch.load(SAVE_MODEL_PATH))
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
embedding.weight 	 torch.Size([4087, 1024])
lstm.weight_ih_l0 	 torch.Size([2048, 1024])
lstm.weight_hh_l0 	 torch.Size([2048, 512])
lstm.bias_ih_l0 	 torch.Size([2048])
lstm.bias_hh_l0 	 torch.Size([2048])
lstm.weight_ih_l1 	 torch.Size([2048, 512])
lstm.weight_hh_l1 	 torch.Size([2048, 512])
lstm.bias_ih_l1 	 torch.Size([2048])
lstm.bias_hh_l1 	 torch.Size([2048])
fc.weight 	 torch.Size([5, 512])
fc.bias 	 torch.Size([5])


## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [38]:
import numpy as np

def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    # Convert words to ids
    print(text)
    tokens = preprocess(text)
    print(tokens)
    
    # Filter non-vocab words
    tokens = [token for token in tokens if token in vocab]
    print(tokens)
    
    # Convert words to ids
    tokens =  [vocab[token] for token in tokens]
    print(tokens)
       
    # Adding a batch dimension
    token_tensor = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1, 1)))
    print(token_tensor)
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(token_tensor, hidden)
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred


In [39]:
text = "Google is working on self driving cars, I'm bullish on $goog"

model.eval()

model.to("cpu")
predict(text, model, vocab)

Google is working on self driving cars, I'm bullish on $goog
['google', 'be', 'work', 'on', 'self', 'drive', 'cars', 'bullish', 'on']
['google', 'work', 'self', 'drive', 'cars', 'bullish']
[3037, 20, 2705, 2953, 1487, 1042]
tensor([[3037.],
        [  20.],
        [2705.],
        [2953.],
        [1487.],
        [1042.]])


tensor([[0.0612, 0.0278, 0.4305, 0.3736, 0.1069]], grad_fn=<ExpBackward>)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**
If the array is arranged as review_rating label from 4 to 0 (with 4 is the best rating), there is 25% probability it is the 4.

There are also a few things that i hope that i can get your suggestions and comments.

1. What is the best way to set the low_cutoff. I set it at 1e-5, which i am still not quite sure if it is the optimal one.

2. My model trained with 2 epochs didn't seem to have significant improvement on lower training loss. Is there anything wrong with my classifier and training model set up?

3. For the Making Prediction job, i can only run the prediction with GPU. I am not sure why i cannot run it on the CPU mode. I have posted the question on Knowledge as https://knowledge.udacity.com/questions/30679. Please help

4. How do i know the 5 probabilities at the prediction output belongs to which rating?

Many thanks!



Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [17]:
with open(os.path.join('data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [25]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

# next(twit_stream())

Using the `prediction` function, let's apply it to a stream of twits.

In [26]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [27]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

tensor([[3620.],
        [1549.],
        [3138.],
        [2911.],
        [1538.],
        [ 655.]])
tensor([[1826.],
        [ 147.],
        [2753.],
        [ 251.],
        [1089.],
        [  36.],
        [ 452.],
        [3505.],
        [2773.],
        [1272.],
        [3526.],
        [3240.],
        [2290.],
        [1245.],
        [3830.]])
tensor([[3149.],
        [3554.]])


{'score': tensor([[4.4095e-06, 6.3261e-01, 1.3331e-03, 3.6427e-01, 1.7863e-03]],
        grad_fn=<ExpBackward>),
 'symbol': '$AAPL',
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.